<a href="https://colab.research.google.com/github/hyoh79/ML/blob/main/SL(cancer_classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import load_breast_cancer
cancer=load_breast_cancer()
print("cancer.keys():\n",cancer.keys())

In [ ]:
print("유방암 데이터의 형태:",cancer.data.shape)
print("유방암 데이터의 형태(또 다른 명령어 표현)", cancer['data'].shape)
print("유방암 데이터의 타입:", type(cancer)) # load_breat_cancer( )가 반환한 cancer 객체는 파이썬 딕셔너리와 유사한 Bunch 클래스의 객체로 키(keys)와 값(value)으로 구성되어 있기 때문

In [ ]:
import numpy as np
print("클래스별 샘플 개수:\n",{n:v for n,v in zip(cancer.target_names,np.bincount(cancer.target))})

In [ ]:
print("특성 이름:\n",cancer.feature_names)

In [ ]:
print("데이터 정보:\n",cancer.DESCR)

In [ ]:
print(cancer.data.shape)
print(cancer['feature_names'])
print(cancer['data'][:5])

In [ ]:
print(cancer['target'])
print(type(cancer['target']))
print(cancer.target.shape)

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, stratify=cancer.target, random_state=66)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cancer['data'],cancer['target'],stratify=cancer.target,random_state=66)

In [ ]:
training_accuracy=[]
test_accuracy=[]

neighbors_settings=range(1,11) #1에서 10까지 n_neighbors를 적용

## [Algorithm#1: kNN for classification]
from sklearn.neighbors import KNeighborsClassifier
for n_neighbors in neighbors_settings:
  clf=KNeighborsClassifier(n_neighbors=n_neighbors)
  clf.fit(X_train, y_train)
  training_accuracy.append(clf.score(X_train,y_train))
  test_accuracy.append(clf.score(X_test,y_test))

In [ ]:
import matplotlib.pylab as plt
plt.plot(neighbors_settings,training_accuracy,label="Trainig Accuracy")
plt.plot(neighbors_settings, test_accuracy, label="Test Accuracy")
plt.ylabel("accuracy")
plt.xlabel("# of neighbors")
plt.legend()
# 이웃을 하나 선택했을때 결정경계가 훈련 데이터에 가깝게 따라감(복잡한 모델, 과대적합). 
# 이웃의 수를 늘릴수록 결정 경계는 더 부드러워짐(단순한 모델을 의미).
# cancer dataset으로 kNN에서 이웃의 수를 변화시켜보면서 모델의 복잡도와 일반화 사이의 관계를 입증해볼 수 있음.

In [ ]:
## [Algorithm#2: Logistic Regression for classification]

from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(max_iter=5000).fit(X_train,y_train)

print("훈련 세트 점수: {:.3f}".format(logreg.score(X_train, y_train)))
print("테스트 세트 점수: {:.3f}".format(logreg.score(X_test,y_test)))
# 기본값 C=1이 훈련세트와 테스트 세트 양쪽에 95% 정확도로 꽤 훌륭한 성능을 내고 있지만
# 훈련세트와 테스트 세트의 성능이 매우 비슷 (과소적합)--> 해결책(모델의 제약/규제을 더 풀어주기 위해 C를 증가시켜볼것!)

In [ ]:

logreg100=LogisticRegression(C=100,max_iter=5000).fit(X_train,y_train) # C를 증가시킴(복잡도를 높인 모델, 성능이 좋음)

print("훈련 세트 점수: {:.3f}".format(logreg.score(X_train, y_train)))
print("테스트 세트 점수: {:.3f}".format(logreg.score(X_test,y_test)))

In [ ]:

logreg001=LogisticRegression(C=0.01,max_iter=50000).fit(X_train,y_train) 
# C를 감소시킴.(규제를 높인 모델, 이미 과소적합된 모델에서 규제를 더 높였기에 기본 매개변수 C=1일때보다 성능이 낮아짐)

print("훈련 세트 점수: {:.3f}".format(logreg.score(X_train, y_train)))
print("테스트 세트 점수: {:.3f}".format(logreg.score(X_test,y_test)))

In [ ]:
import matplotlib.pyplot as plt
plt.plot